# [실습 24-1] 시퀀스-투-시퀀스 모델(Seq2Seq Model)


# 자연어 처리와 시퀀스 모델
이번 장(24장)에서는 시퀀스-투-시퀀스 모델에 대한 실습을 진행해 보도록 하겠습니다.

시퀀스-투-시퀀스 모델은 시퀀스가 입력으로 들어가면 출력 또한 시퀀스로 출력되는 모델을 말합니다.

이번 시간에는 케라스(Keras) 공식 블로그에 나와있는 모델을 바탕으로, 영어-프랑스어 간 번역모델을 간단하게 구현해보고 그 원리를 이해해 보도록 합시다.

# 실습
1. (입력 데이터 수, 입력 최대 길이, 토큰 수) 크기의 값이 0인 배열을 생성합니다.

2. LSTM층을 변수 encoder에 저장합니다.(return_state=True / latent_dim 활용)

3. LSTM층을 변수 decoder에 저장합니다.(return_state=True, return_sequence=True / latent_dim 활용)

4. inital_state를 인코더의 states로 설정하고 디코더의 입력값을 바탕으로 lstm 연산을 수행하는 층을 만들어 줍니다.

In [1]:
from __future__ import print_function

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 128
num_samples = 1000
data_path = './data/fra.txt'

input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

## 데이터를 불러옵니다.
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

## num_samples개수의 데이터에 대해서 "\t" 기호를 바탕으로 토큰화합니다. 
for line in lines[: min(num_samples, len(lines) - 1)]:

    input_text, target_text = line.split('\t')
    # TODO: target_text에 대해서 (앞)tap(\t) 과 (뒤)'\n' 추가하여 right shifted 되도록 합니다. #전체 하나씩 돌면서 #list를 씌워주면 token단위로 뽑아진다. 즉 분석 단위가 바귐. 
    target_text = '\t' + target_text +'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    #input_text의 글자(char)에 대해서 input_characters에 없다면 추가해줍니다. 
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    #target_text의 글자(char)에 대해서 target_characters에 없다면 추가해줍니다.
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

# Characteristic 수준에서의 토큰화

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
enc_num_tokens = len(input_chars)  # input의 character별로 모아놓은 것의 token 개수(len)
dec_num_tokens = len(target_chars) # target의 character별로 모아놓은 것의 token 개수(len)
#(인코더, 디코더) 입력 문장의 최대 길이를 입력 시퀀스의 최대 길이로 설정합니다. 
enc_max_len = max([len(text) for text in input_texts])
dec_max_len = max([len(text) for text in target_texts])

print('샘플의 개수:', len(input_texts))
print('중복되지 않는 입력의 개수:', enc_num_tokens)
print('중복되지 않는 출력의 개수:', dec_num_tokens)
print('입력 시퀀스의 최대 길이:', enc_max_len)
print('출력 시퀀스의 최대 길이:', dec_max_len)

#입력과 타겟 데이터의 {'token':index } 딕셔너리를 생성합니다.
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

#TODO: (입력 데이터 수, 입력 최대 길이, 토큰 수) 크기의 값이 0인 배열을 생성합니다.(초기화) #여기 다 디코더에 관한 것.
enc_input_data = np.zeros((len(input_texts),enc_max_len, enc_num_tokens), dtype='float32') #초기값이라서 zero를 넣은 것.
dec_input_data = np.zeros((len(input_texts),dec_max_len, dec_num_tokens),dtype='float32')
dec_target_data = np.zeros((len(input_texts),dec_max_len,dec_num_tokens),dtype='float32')


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    #enc_input_data > (data length, sequence length, total_word)
    for t, char in enumerate(input_text):
        enc_input_data[i, t, input_token_index[char]] = 1.
    
    #dec_input_data > (data length, sequence length, total_word)
    for t, char in enumerate(target_text):
        dec_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            dec_target_data[i, t - 1, target_token_index[char]] = 1.
            

#인코더의 입력층을 설정하여 줍니다. 
enc_inputs = Input(shape=(None, enc_num_tokens))
# TODO: LSTM층을 변수 encoder에 저장합니다.(return_state=True / latent_dim 활용) #LSTM위에 정의되어있음.
encoder = LSTM(latent_dim,return_state=True) # 우리가 정한 max length의 sequence 차원의 vector를 그대로 출력하느냐 아니면 하나의 값만 출력하느냐/ #return_state=True->hidden state값을 출력하느냐 마느냐
enc_outputs, state_h, state_c = encoder(enc_inputs)
encoder_states = [state_h, state_c]

#디코더의 입력층을 설정하여 줍니다.
dec_inputs = Input(shape=(None, dec_num_tokens))
# TODO: LSTM층을 변수 decoder에 저장합니다.(return_state=True, return_sequences=True / latent_dim 활용) 
dec_lstm = LSTM(latent_dim,return_state=True, return_sequences=True )

# TODO:inital_state를 인코더의 states로 설정하고 디코더의 입력값을 바탕으로 lstm 연산을 수행하는 층을 만들어 줍니다.
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=encoder_states)
dec_dense = Dense(dec_num_tokens, activation='softmax')
dec_outputs = dec_dense(dec_outputs)

# 인코더와 디코더를 바탕으로 모델을 선언합니다.
model = Model([enc_inputs, dec_inputs], dec_outputs)

# 모델의 파라미터를 설정하고 학습을 진행합니다.
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([enc_input_data, dec_input_data], dec_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

enc_model = Model(enc_inputs, encoder_states)

dec_state_input_h = Input(shape=(latent_dim,))
dec_state_input_c = Input(shape=(latent_dim,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]
dec_outputs, state_h, state_c = dec_lstm(dec_inputs, initial_state=dec_states_inputs)

decoder_states = [state_h, state_c]
dec_outputs = dec_dense(dec_outputs)
decoder_model = Model(
    [dec_inputs] + dec_states_inputs,
    [dec_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # 인코더 모델을 바탕으로 input_seq에 대한 state_value를 예측합니다.
    states_value = enc_model.predict(input_seq)
    # (1,1, 디코더 토큰 개수) 크기를 가진 값이 0인 배열을 만들어 줍니다.
    seq_target = np.zeros((1, 1, dec_num_tokens))
    seq_target[0, 0, target_token_index['\t']] = 1.

    _stop = False
    _decoded = ''
    while not _stop:
        output_tokens, h, c = decoder_model.predict(
            [seq_target] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        _decoded += sampled_char
        # 디코딩된 글자의 수가 dec_max_len을 넘거나 그 글자가 개행('\n')이면 _stop을 True로 활성화시킵니다. 
        if (sampled_char == '\n' or
           len(_decoded) > dec_max_len):
            _stop = True
        seq_target = np.zeros((1, 1, dec_num_tokens))
        seq_target[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return _decoded


for seq_index in range(100):
    input_seq = enc_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

FileNotFoundError: [Errno 2] No such file or directory: './data/fra.txt'

# [실습 24-2] 어텐션 매커니즘(Attention Mechanism)


# 어텐션 매커니즘
어텐션 매커니즘(Attention Mechanism)은 기존의 가중치(weight)와 달리 데이터의 특징 또는 형태를 반영하여 도출한 어텐션 가중치(attention weight)를 바탕으로 일련의 연산을 수행하는 것을 의미합니다.

어텐션 매커니즘은 그 절차 및 방식에 따라, Additive Attention과 Dot-product Attention으로 나누어 볼 수 있습니다.

이번 시간에는 Additive Attention을 코드를 통해 구현해보고 그 특징을 이해해 봅시다.

# 실습
1. _value, _query에 대해 같은 출력을 내보내는 FC 레이어를 생성합니다.

2. _query의 차원을 추가시킵니다.(axis=1)

3. softmax함수를 통과한 attention value 값을 attention_weight에 저장합니다.

4. attention_weight과 _value를 곱한 값의 요소별 평균(axis=1)을 context_vector에 저장합니다.

5. 간단한 차원의 값들을 조정해가며 어텐션 매커니즘을 직관적으로 이해해봅시다.

In [2]:
import tensorflow as tf

def attention(_query, _value):
    ACT='tanh'
    weights={
        # TODO: _value, _query에 대해 같은 출력을 내보내는 FC 레이어를 생성합니다.
        'value' : tf.keras.layers.Dense(10, activation=ACT), #value랑query값이 같게 맞춰. 더해질 때 에러안 나게.
        'query' : tf.keras.layers.Dense(10, activation=ACT),
        'projection' : tf.keras.layers.Dense(1, activation=ACT)
        }
    # TODO: _query의 차원을 추가시킵니다.(axis=1)
    _query = tf.expand_dims(_query,1)

    attention_value = weights['projection'](weights['value'](_value)+weights['query'](_query))

    # TODO: softmax함수를 통과한 attention value 값을 attention_weight에 저장합니다.
    attention_weight = tf.nn.softmax(attention_value,axis=1)
    # TODO: attention_weight과 _value를 곱한 값의 요소별 평균(axis=1)을 context_vector에 저장합니다.
    context_vector = tf.reduce_sum(attention_weight*_value,axis=1)
    
    return attention_weight, context_vector
    
def print_fn(attention_weight, context_vector):
    print("어텐션 가중치:")
    print(attention_weight.numpy())
    print("컨텍스트 벡터:")
    print(context_vector.numpy())

#TODO: 간단한 차원의 값들을 조정해가며 어텐션 매커니즘을 직관적으로 이해해봅시다.

print("--------------------------case1--------------------------")
_value = tf.constant([[[0],[0],[0]]], dtype = tf.float32)
_query = tf.constant([[0]], dtype = tf.float32)

attention_weight, context_vector = attention(_query, _value) 
print_fn(attention_weight, context_vector)

print("--------------------------case2--------------------------")
_value = tf.constant([[[0],[0],[0]]], dtype = tf.float32)
_query = tf.constant([[0, 0, 0]], dtype = tf.float32)

attention_weight, context_vector = attention(_query, _value) 
print_fn(attention_weight, context_vector)

print("--------------------------case3--------------------------")
_value = tf.constant([[[0],[1],[0]]], dtype = tf.float32)
_query = tf.constant([[0],[0]], dtype = tf.float32)

attention_weight, context_vector = attention(_query, _value) 
print_fn(attention_weight, context_vector)

print("--------------------------case4--------------------------")
_value = tf.constant([[[0],[1],[0]]], dtype = tf.float32)
_query = tf.constant([[1],[0]], dtype = tf.float32)

attention_weight, context_vector = attention(_query, _value) 
print_fn(attention_weight, context_vector)


--------------------------case1--------------------------
어텐션 가중치:
[[[0.33333334]
  [0.33333334]
  [0.33333334]]]
컨텍스트 벡터:
[[0.]]
--------------------------case2--------------------------
어텐션 가중치:
[[[0.33333334]
  [0.33333334]
  [0.33333334]]]
컨텍스트 벡터:
[[0.]]
--------------------------case3--------------------------
어텐션 가중치:
[[[0.25081095]
  [0.49837807]
  [0.25081095]]

 [[0.25081095]
  [0.49837807]
  [0.25081095]]]
컨텍스트 벡터:
[[0.49837807]
 [0.49837807]]
--------------------------case4--------------------------
어텐션 가중치:
[[[0.2632927 ]
  [0.4734145 ]
  [0.2632927 ]]

 [[0.23611811]
  [0.5277638 ]
  [0.23611811]]]
컨텍스트 벡터:
[[0.4734145]
 [0.5277638]]


# [실습 24-3] 어텐션을 활용한 신경망 기계 번역(Neural Machine Translation)


# 신경망 기계 번역
  신경망 기계 번역(Neural Machine Translation)은 서로 다른 언어의 문장에 대한 번역을 인공 신경망 알고리즘을 통해 구현한 것을 의미합니다.

   이번 시간에는 지난 시간에 알아본 Seq2Seq 모델과 Attention 알고리즘을 이용하여 간단한 신경망 기계번역을 구현하여 봅시다.

   데이터 셋은 영어(English)-스페인어(Spanish) 번역 데이터입니다.

(실습은 텐서 플로우 공식 튜토리얼을 바탕으로 작성되었습니다.)

# 실습
1. GRU 신경망을 Keras를 이용하여 설정한 인코더 유닛만큼 self.gru에 선언합니다.(return_sequences=True, return_state=True)

2. GRU의 입력을 x로, 초기 상태를 hidden으로 설정합니다.

3. values를 W1에 _hidden을 W2에 연산을 적용한 후, 이를 V에 적용합니다.

In [3]:
import tensorflow as tf
import pickle, os
from sklearn.model_selection import train_test_split

DATA_PATH = './data/'
BATCH_SIZE = 32
EMD_DM = 128
UNITS = 128
EPOCHS = 1

def load_data(path):
    with open(path + 'input.pickle', 'rb') as handle:
        _input = pickle.load(handle)
    with open(path + 'label.pickle', 'rb') as handle:
        _label = pickle.load(handle)
    return _input, _label

def load_tokenizer(path):
    with open(path + 'tokenizer_input.pickle', 'rb') as handle:
        _input = pickle.load(handle)
    with open(path + 'tokenizer_target.pickle', 'rb') as handle:
        _target = pickle.load(handle)
    return _input, _target

#데이터와 토크나이저를 불러옵니다.
_input, _label = load_data(DATA_PATH)
lan_input, lan_target = load_tokenizer(DATA_PATH)

input_train, input_test, label_train, label_test = train_test_split(_input, _label, test_size=.2)

BUFFER_SIZE = len(input_train)
steps_per_epoch = len(input_train) // BATCH_SIZE
SIZE_INPUT = len(lan_input.word_index) + 1
SIZE_TARGET = len(lan_target.word_index) + 1

# tf data를 사용하여 데이터셋을 다루어 봅니다.
dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

#인코더를 정의하는 클래스 입니다.
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, EMD_DM, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, EMD_DM)
        # TODO: GRU 신경망을 Keras를 이용하여 설정한 인코더 유닛만큼 self.gru에 선언합니다.(return_sequences=True, return_state=True)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        x = self.embedding(x)
        # TODO: GRU의 입력을 x로, 초기 상태를 hidden으로 설정합니다. 
        output, state = self.gru(x)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

#어텐션을 정의하는 클래스 입니다.
class Attention(tf.keras.Model):
    def __init__(self, UNITS):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(UNITS)
        self.W2 = tf.keras.layers.Dense(UNITS)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):       
        _hidden = tf.expand_dims(query, 1)
        # TODO: values를 W1에 _hidden을 W2에 연산을 적용한 후, 이를 V에 적용합니다. 
        score = self.V(tf.nn.tanh(self.W1( values) + self.W2(_hidden)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# 디코더를 정의하는 클래스 입니다.
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, EMD_DM, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, EMD_DM)
        self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)

    def call(self, _x, hidden, out_encoder):
        context_vector, attention_weights = self.attention(hidden, out_encoder)
        _x = self.embedding(_x)
        _x = tf.concat([tf.expand_dims(context_vector, 1), _x], axis=-1)
        output, state = self.gru(_x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state, attention_weights

#손실함수를 정의하는 함수입니다.
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


def train_step(input_, target_, hidden_encoder):
    loss = 0

    with tf.GradientTape() as tape:
        output_encoder, hidden_encoder = encoder(input_, hidden_encoder)
        hidden_decoder = hidden_encoder
        input_decoder = tf.expand_dims([lan_target.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, target_.shape[1]):
            predictions, hidden_decoder, _ = decoder(input_decoder, hidden_decoder, output_encoder)
            loss += loss_function(target_[:, t], predictions)
            dec_input = tf.expand_dims(target_[:, t], 1)

    loss_batch = (loss / int(target_.shape[1]))
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss_batch


encoder = Encoder(SIZE_INPUT, EMD_DM, UNITS, BATCH_SIZE)
decoder = Decoder(SIZE_TARGET, EMD_DM, UNITS, BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam()

# 학습을 진행합니다.
for epoch in range(EPOCHS):
    hidden_encoder = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (input_data, target_data)) in enumerate(dataset.take(steps_per_epoch)):

        batch_loss = train_step(input_data, target_data, hidden_encoder)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1, batch, batch_loss.numpy()))
    print("[EPOCH - {}] Total Loss: {}".format(epoch+1, total_loss/steps_per_epoch))

FileNotFoundError: [Errno 2] No such file or directory: './data/input.pickle'

# [실습 24-4] 간단한 챗봇(Chat-bot) 만들기


# 간단한 챗봇 만들기
이번 시간에는 시퀀스 모델이 가장 많이 활용되는 분야 중 하나인 챗봇(Chat-bot)에 대해 살펴보도록 하겠습니다.

영어 자연어처리 관련 라이브러리인 nltk를 활용하여 간단한 응답을 출력하는 모델을 만들어 봅니다.

# 실습
1. ‘!"#$%&’()+, -./:;<=>?@[]^{|}~’ 를 딕셔너리 remove_punct_dict로 저장합니다.(string.punctuation 활용)

2. TfidfVectorizer를 생성합니다.

3. tfidf가 적용된 user_responce (tfidf[-1])과 전체 값을 바탕으로 코사인 유사도를 구합니다.

4. GREETING_INPUTS 에 있는 값을 입력했을 때, GREETING_RESPONSES 이 나오는지 확인해 봅시다.

In [4]:
import io, random, string 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) 

nltk.download('punkt') 
nltk.download('wordnet') 

def load_data():
    with open('./data/chatbot.txt','r', encoding='utf8', errors ='ignore') as fin:
        raw = fin.read().lower()
    return raw
    
raw = load_data()
sent_tokens = nltk.sent_tokenize(raw) 
word_tokens = nltk.word_tokenize(raw)

# WordNetLemmatizer를 선언합니다.
lemmer = WordNetLemmatizer() #먹었습니다를 받아서 먹었다 라고 즉 기본형으로 바꿔주는 것.
LemTokens = lambda tokens: [lemmer.lemmatize(token) for token in tokens]#토큰을 받아서
LemNormalize = lambda text: LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) #불용어 제거

# TODO: '!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~'를 딕셔너리 remove_punct_dict로 저장합니다.(string.punctuation 활용)#string은 내장함수 # string에서 for문이돌면 chararcter가 하나씩 돌기 때문에 character하나씩 나와서 숫자로 바꿔서 dictionary에 저장하고
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation )


# 키워드 매칭에 사용될 입력과 응답입니다.(테스트 시, 여기있는 입력값을 입력하면 대응되는 출력값이 출력됩니다.) 
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey","hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    # 인사말이 입력되면, 그에 맞는 응답을 랜덤하게 반환합니다.
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


# 응답(Response)을 생성하는 함수입니다.
def response(user_response):
    robot_response=''
    sent_tokens.append(user_response)
    #TODO: TfidfVectorizer를 생성합니다.(LemNormalize 활용)
    TfidfVec = TfidfVectorizer(LemNormalize)
    # sent_token을 Tfidf 값으로 변환합니다.
    tfidf = TfidfVec.fit_transform(sent_tokens)
    # TODO: tfidf화 된 user_responce (tfidf[-1])과 전체 값을 바탕으로 코사인 유사도를 구합니다.
    vals = cosine_similarity(user_response,dense_output=True)
    
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf==0):
        robot_response = robot_response+"I am sorry! I don't understand you"
    else:
        robot_response = robot_response+sent_tokens[idx]
    
    return robot_response

# 학습을 진행합니다. #실제 실행창
flag=True
print("ROBOT: If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBOT: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBOT: "+greeting(user_response))
            else:
                print("ROBOT: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yein4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yein4\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: './data/chatbot.txt'

# [미션 24] 신경망 기계 번역의 성능 높이기



# 미션! 신경망 기계번역 성능 높이기
이번 시간에는 지난 [실습 24-3]에서 구현했던 신경망 기계번역 모델을 수정하여 그 성능을 높여보도록 합시다.

옆의 코드는 [실습 24-3]의 코드를 그대로 옮겨 놓았습니다. 해당 실습에서 완성하였던 코드를 옮겨 적으신 후, 모델 내부의 수정을 통해 총 손실값(total_loss/steps_per_epoch)가 2.3 이하가 되도록 해보세요.

(모델 수정은 지금까지 배웠던 여러 신경망들을 더하거나 파라미터, 하이퍼파라미터를 변경함으로써 달성할 수 있습니다.)

# 미션
1. [실습 24-3] 에서 빈칸을 채운 모델을 복사하여 옮겨줍니다.

2. Encoder, Decoder, Attention 클래스 또는 파라미터를 수정하여 total_loss가 2.3이하가 되도록 합니다.(EPOCHS=1로 고정합니다)

In [5]:
import tensorflow as tf
import pickle, os
from sklearn.model_selection import train_test_split

DATA_PATH = './data/'
BATCH_SIZE = 32
EMD_DM = 128
UNITS = 128
EPOCHS = 1

def load_data(path):
    with open(path + 'input.pickle', 'rb') as handle:
        _input = pickle.load(handle)
    with open(path + 'label.pickle', 'rb') as handle:
        _label = pickle.load(handle)
    return _input, _label

def load_tokenizer(path):
    with open(path + 'tokenizer_input.pickle', 'rb') as handle:
        _input = pickle.load(handle)
    with open(path + 'tokenizer_target.pickle', 'rb') as handle:
        _target = pickle.load(handle)
    return _input, _target

#데이터와 토크나이저를 불러옵니다.
_input, _label = load_data(DATA_PATH)
lan_input, lan_target = load_tokenizer(DATA_PATH)

input_train, input_test, label_train, label_test = train_test_split(_input, _label, test_size=.2)

BUFFER_SIZE = len(input_train)
steps_per_epoch = len(input_train) // BATCH_SIZE
SIZE_INPUT = len(lan_input.word_index) + 1
SIZE_TARGET = len(lan_target.word_index) + 1

# tf data를 사용하여 데이터셋을 다루어 봅니다.
dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

#인코더를 정의하는 클래스 입니다.
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, EMD_DM, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, EMD_DM)
        # TODO: GRU 신경망을 Keras를 이용하여 설정한 인코더 유닛만큼 self.gru에 선언합니다.(return_sequences=True, return_state=True)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        # TODO: GRU의 입력을 x로, 초기 상태를 hidden으로 설정합니다. 
        output, state = self.gru(x)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

#어텐션을 정의하는 클래스 입니다.
class Attention(tf.keras.Model):
    def __init__(self, UNITS):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(UNITS)
        self.W2 = tf.keras.layers.Dense(UNITS)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):       
        _hidden = tf.expand_dims(query, 1)
        # TODO: values를 W1에 _hidden을 W2에 연산을 적용한 후, 이를 V에 적용합니다. 
        score = self.W1( values)*self.W2(_hidden)
        #self.V(self.W1( values)*self.W2(_hidden))
        #self.V(self.W1( values) +self.W2(_hidden))
        #self.V(tf.nn.tanh(self.W1( values) + self.W2(_hidden)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# 디코더를 정의하는 클래스 입니다.
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, EMD_DM, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, EMD_DM)
        self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)

    def call(self, _x, hidden, out_encoder):
        context_vector, attention_weights = self.attention(hidden, out_encoder)
        _x = self.embedding(_x)
        _x = tf.concat([tf.expand_dims(context_vector, 1), _x], axis=-1)
        output, state = self.gru(_x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state, attention_weights

#손실함수를 정의하는 함수입니다.
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


def train_step(input_, target_, hidden_encoder):
    loss = 0

    with tf.GradientTape() as tape:
        output_encoder, hidden_encoder = encoder(input_, hidden_encoder)
        hidden_decoder = hidden_encoder
        input_decoder = tf.expand_dims([lan_target.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, target_.shape[1]):
            predictions, hidden_decoder, _ = decoder(input_decoder, hidden_decoder, output_encoder)
            loss += loss_function(target_[:, t], predictions)
            dec_input = tf.expand_dims(target_[:, t], 1)

    loss_batch = (loss / int(target_.shape[1]))
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss_batch
    
    
encoder = Encoder(SIZE_INPUT, EMD_DM, UNITS, BATCH_SIZE)
decoder = Decoder(SIZE_TARGET, EMD_DM, UNITS, BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam()

def main():
    total_score=0
    for epoch in range(EPOCHS):
        hidden_encoder = encoder.initialize_hidden_state()
        total_loss = 0
        for (batch, (input_data, target_data)) in enumerate(dataset.take(steps_per_epoch)):

            batch_loss = train_step(input_data, target_data, hidden_encoder)
            total_loss += batch_loss
            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
        print("[EPOCH - {}] Total Loss: {}".format(epoch + 1, total_loss/steps_per_epoch))
        total_score=total_loss/steps_per_epoch
    return total_score
    
if __name__=="__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: './data/input.pickle'